In [1]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import pandas as pd
import numpy as np
import json
import pickle
import os
import joblib
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result/{}/{}_*.csv"

In [11]:
%load_ext autoreload
%autoreload 2

from src import classifiers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def create_directories(path):
    
    if not os.path.exists(path):
        os.makedirs(path)
        os.makedirs("{}/summaries".format(path))
        os.makedirs("{}/predictions".format(path))

## Load data

In [13]:
with open('autoencoder_64dim/dataset_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

In [14]:
with open('mv_tunning/parameters.pkl', 'rb') as fp:
    parameters = pickle.load(fp)

## Train Models

In [15]:
tests = {'mv_models_64dim':'mv'}

In [16]:
sections=['introduction', 'materials', 'conclusion']
name_models = ['mlp']

In [17]:
def main(dataset, parameters, sections, name_models, num_test):
    
    for name_test, approach in tests.items():
    
        if approach == 'sv':
            index_Xtrain=0
            index_ytrain=2
        elif approach == 'mv':
            index_Xtrain=-2
            index_ytrain=2

        print(approach)

        path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(name_test)
        create_directories(path_to_save)
        
        if not os.path.exists("{}/test_{}".format(path_to_save, num_test)):
            os.makedirs("{}/test_{}".format(path_to_save, num_test))

        for name_model in name_models: 

            classifiers.pipeline_classifiers(
                dataset, parameters, sections, [name_model], index_Xtrain, index_ytrain, num_test, path_to_save)

In [ ]:

from joblib import Parallel, delayed

Parallel(n_jobs=5)(delayed(main)(dataset, parameters, sections, name_models, num_test) for num_test in range(1, 31))
 